# Assignment 6: Hidden Markov Models

## Submission
All submissions will be via Gradescope. If you're completing this assignment in Jupyter Notebook, you must run the `notebook2script.py` file to export your work to a python file. To generate your submission file, run the command 

`python notebook2script.py submission`

and your file will be created under the `submission` directory.

Upload the resulting `submission.py` file to the Assignment 6 assignment on Gradescope for feedback.

#### IMPORTANT: A total of 10 submissions is allowed for this assignment. Please use your submissions carefully and do not submit until you have thoroughly tested your code locally.

#### If you're at 9 submissions, use your tenth and last submission wisely. The submission marked as ‘Active’ in Gradescope will be the submission counted towards your grade. 

Please also submit your submission.py to Canvas as backup.



## Overview

The goal of this assignment is to demonstrate the power of probabalistic models. You will build a word recognizer for American Sign Language (ASL) video sequences. In particular, this project employs hidden Markov models (HMM's) to analyze a series of measurements taken from videos of American Sign Language (ASL) collected for research (see the [RWTH-BOSTON-104 Database](http://www-i6.informatik.rwth-aachen.de/~dreuw/database-rwth-boston-104.php)).

In each video, an ASL signer is signing a meaningful sentence. In a typical ASL recognition system, you observe the XY coordinates of the speaker's left hand, right hand, and nose for every frame. The following diagram shows how the positions of the left hand (Red), right hand (Blue), and nose (Green) change over time in video number #66. Saturation of colors represents time elapsed.

<img src="./demo/hands_nose_position.png" alt="hands nose position">

In this assignment, for the sake of simplicity, you will only use the Y-coordinates of each hand to construct your HMM. In Part 1 you will build a one dimensional model, recognizing words based only on a series of right-hand Y coordinates; in Part 2 you will go multidimensional and utilize both hands. At this point, you will have two observed coordinates at each time step (frame) representing right hand & left hand Y positions.

The words you will be recognizing are "BUY", "HOUSE", and "CAR". These individual signs can be seen in the sign phrases from our dataset:

![](demo/buy_house_slow.gif) 

<p style="text-align:center; font-weight:bold"> JOHN CAN BUY HOUSE </p> 

![](demo/buy_car_slow.gif) 

<p style="text-align:center;  font-weight:bold"> JOHN BUY CAR [FUTURE] </p> 

### Part 1a: Encoding the HMM
_[15 Points]_

Follow the method taught on Udacity **Lecture 8: 29. HMM Training** to determine following values for each word:
1. the transition probabilities of each state
2. the mean & standard deviation of emission Gaussian distribution of each state

Use the training samples from the table below. Provide the transition, prior, and emission probabilities parameters for all three words with **accuracy to 3 decimal digits**.

Round the values to 3 decimal places thoughout entire assignment:
- 0.1 stays 0.1 or 0.100
- 0.1234 rounds to 0.123
- 0.2345 rounds to 0.235
- 0.3456 rounds to 0.346
- 0.0123 rounds to 0.012
- 0.0125 rounds to 0.013

Those values can be hardcoded in your program. Don't use round() from python.

Word | Frames | Observed sequence | Initial State1 | Initial State2 | Initial State3
--- | --- | --- | --- | --- | --- 
BUY | 6 | 36, 44, 52, 56, 49, 44 | 36, 44 | 52, 56 | 49, 44
BUY | 8 | 42, 46, 54, 62, 68, 65, 60, 56 | 42, 46, 54 | 62, 68, 65 | 60, 56
BUY | 10 | 42, 40, 41, 43, 52, 55, 59, 60, 55, 47 | 42, 40, 41|43, 52, 55|59, 60, 55, 47
CAR | 10 | 47, 39, 32, 34, 36, 42, 42, 42, 34, 25|47, 39, 32|34, 36, 42|42, 42, 34, 25
CAR | 9 | 35, 35, 43, 46, 52, 52, 56, 49, 45|35, 35, 43|46, 52, 52|56, 49, 45
CAR | 8 | 28, 35, 46, 46, 48, 43, 43, 40|28, 35, 46|46, 48, 43|43, 40
HOUSE| 15 | 37, 36, 32, 26, 26, 25, 23, 22, 21, 39, 48, 60, 70, 74, 77|37, 36, 32, 26, 26|25, 23, 22, 21, 39|48, 60, 70, 74, 77
HOUSE| 15 | 50, 50, 49, 47, 39, 39, 38, 38, 50, 56, 61, 67, 67, 67, 67|50, 50, 49, 47, 39|39, 38, 38, 50, 56|61, 67, 67, 67, 67
HOUSE| 16 | 45, 43, 44, 43, 40, 35, 36, 37, 39, 45, 60, 68, 66, 72, 72, 75|45, 43, 44, 43, 40|35, 36, 37, 39, 45|60, 68, 66, 72, 72, 75

As shown in the diagram below, each one of the three words (BUY, CAR, and HOUSE) has exactly **THREE hidden states** in its HMM. All words have equal probability of occuring. Modify the prior accordingly. All words must start from State 1 and can only transit to the next state or stay in the current one as shown below:

<img src="part_1_a_probs.png">

### _Training sequences need to have 3 hidden states no matter what!_
If you follow the procedure on the Udacity lecture video, you might encounter a situation where a hidden state is **_squeezed_** out by an adjacent state. In that situation, always keep at least one observation for that hidden state.

Example:
Assume you've reached a stage where the following is true: 
- State 1 has mean=53 & std=6
- State 2 has mean=37 & std=9
- State 3 has mean=70 & std=8

The next training sample has the following observed sequence:

`45 45 34 | 30 30 25 36 52 | 62 69 74` 

and you are trying to adjust the location of state boundary between State 1 & 2. You first move it 1 step to the left since 34 is closer to State 2, and then you realize that 45 is still closer to State 2. If you follow the same routine, you will end up with no obvervation for State 1. In order to prevent this from happening, you have to stop at the last "45" and as a result leave the boundary as 

`45 | 45 34 30 30 25 36 52 | 62 69 74`

Now you meet the '3 hidden states per sample' requirement.

In [ ]:
import hmm_submission_test as tests

In [ ]:
#export
import numpy as np
import operator

In [ ]:
#export
def part_1_a():
    """Provide probabilities for the word HMMs outlined below.
    Word BUY, CAR, and HOUSE.
    Review Udacity Lesson 8 - Video #29. HMM Training
    Returns:
        tuple() of
        (prior probabilities for all states for word BUY,
         transition probabilities between states for word BUY,
         emission parameters tuple(mean, std) for all states for word BUY,
         prior probabilities for all states for word CAR,
         transition probabilities between states for word CAR,
         emission parameters tuple(mean, std) for all states for word CAR,
         prior probabilities for all states for word HOUSE,
         transition probabilities between states for word HOUSE,
         emission parameters tuple(mean, std) for all states for word HOUSE,)
        Sample Format (not complete):
        (
            {'B1': prob_of_starting_in_B1, 'B2': prob_of_starting_in_B2, ...},
            {'B1': {'B1': prob_of_transition_from_B1_to_B1,
                    'B2': prob_of_transition_from_B1_to_B2,
                    'B3': prob_of_transition_from_B1_to_B3,
                    'Bend': prob_of_transition_from_B1_to_Bend},
             'B2': {...}, ...},
            {'B1': tuple(mean_of_B1, standard_deviation_of_B1),
             'B2': tuple(mean_of_B2, standard_deviation_of_B2), ...},
            {'C1': prob_of_starting_in_C1, 'C2': prob_of_starting_in_C2, ...},
            {'C1': {'C1': prob_of_transition_from_C1_to_C1,
                    'C2': prob_of_transition_from_C1_to_C2,
                    'C3': prob_of_transition_from_C1_to_C3,
                    'Cend': prob_of_transition_from_C1_to_Cend},
             'C2': {...}, ...}
            {'C1': tuple(mean_of_C1, standard_deviation_of_C1),
             'C2': tuple(mean_of_C2, standard_deviation_of_C2), ...}
            {'H1': prob_of_starting_in_H1, 'H2': prob_of_starting_in_H2, ...},
            {'H1': {'H1': prob_of_transition_from_H1_to_H1,
                    'H2': prob_of_transition_from_H1_to_H2,
                    'H3': prob_of_transition_from_H1_to_H3,
                    'Hend': prob_of_transition_from_H1_to_Hend},
             'H2': {...}, ...}
            {'H1': tuple(mean_of_H1, standard_deviation_of_H1),
             'H2': tuple(mean_of_H2, standard_deviation_of_H2), ...}
        )
    """

    # TODO: complete this function.
    raise NotImplementedError()

    """Word BUY"""
    b_prior_probs = {
        'B1': 0.,
        'B2': 0.,
        'B3': 0.,
        'Bend': 0.,
    }
    b_transition_probs = {
        'B1': {'B1': 0., 'B2': 0., 'B3': 0., 'Bend': 0.},
        'B2': {'B1': 0., 'B2': 0., 'B3': 0., 'Bend': 0.},
        'B3': {'B1': 0., 'B2': 0., 'B3': 0., 'Bend': 0.},
        'Bend': {'B1': 0., 'B2': 0., 'B3': 0., 'Bend': 0.},
    }
    # Parameters for end state is not required
    b_emission_paras = {
        'B1': (None, None),
        'B2': (None, None),
        'B3': (None, None),
        'Bend': (None, None)
    }

    """Word CAR"""
    c_prior_probs = {
        'C1': 0.,
        'C2': 0.,
        'C3': 0.,
        'Cend': 0.,
    }
    c_transition_probs = {
        'C1': {'C1': 0., 'C2': 0., 'C3': 0., 'Cend': 0.},
        'C2': {'C1': 0., 'C2': 0., 'C3': 0., 'Cend': 0.},
        'C3': {'C1': 0., 'C2': 0., 'C3': 0., 'Cend': 0.},
        'Cend': {'C1': 0., 'C2': 0., 'C3': 0., 'Cend': 0.},
    }
    # Parameters for end state is not required
    c_emission_paras = {
        'C1': (None, None),
        'C2': (None, None),
        'C3': (None, None),
        'Cend': (None, None)
    }

    """Word HOUSE"""
    h_prior_probs = {
        'H1': 0.,
        'H2': 0.,
        'H3': 0.,
        'Hend': 0.,
    }
    # Probability of a state changing to another state.
    h_transition_probs = {
        'H1': {'H1': 0., 'H2': 0., 'H3': 0., 'Hend': 0.},
        'H2': {'H1': 0., 'H2': 0., 'H3': 0., 'Hend': 0.},
        'H3': {'H1': 0., 'H2': 0., 'H3': 0., 'Hend': 0.},
        'Hend': {'H1': 0., 'H2': 0., 'H3': 0., 'Hend': 0.},
    }
    # Parameters for end state is not required
    h_emission_paras = {
        'H1': (None, None),
        'H2': (None, None),
        'H3': (None, None),
        'Hend': (None, None)
    }

    return (b_prior_probs, b_transition_probs, b_emission_paras,
            c_prior_probs, c_transition_probs, c_emission_paras,
            h_prior_probs, h_transition_probs, h_emission_paras,)

########## DON'T WRITE ANY CODE OUTSIDE THE FUNCTION! ################
##### CODE BELOW IS USED FOR RUNNING LOCAL TEST DON'T MODIFY IT ######
tests.TestPart1a().test_prior(part_1_a)
tests.TestPart1a().test_b_emission(part_1_a)
tests.TestPart1a().test_c_emission(part_1_a)
tests.TestPart1a().test_h_emission(part_1_a)
tests.TestPart1a().test_b_transition(part_1_a)
tests.TestPart1a().test_c_transition(part_1_a)
tests.TestPart1a().test_h_transition(part_1_a)
################ END OF LOCAL TEST CODE SECTION ######################

### Part 1b: Creating the Viterbi Trellis
_[40 Points]_

The goal here will be to use the HMM derived from Part 1a (states, prior probabilities, transition probabilities, and parameters of emission distribution) to build a viterbi trellis.  When provided with an evidence vector (list of observed right-hand Y coordinates), the function will return the most likely sequence of states that generated the evidence and the probabilty of that sequence being correct.

For example, an evidence vector [36, 44, 52, 53, 49, 44] should output a sequence ['B1', ... 'B2', ... 'B3']

If no sequence can be found, the algorithm should return one of the following tuples:
`(None, 0)` (null),  `([], 0)` (empty list) or  `(['C1', 'C1', ... 'C1'],0)` (Or all being the first state of that letter)

"No sequence can be found" means the probability reaches 0 midway. If you find an incomplete sequence with some probability, output that sequence with its probability. 

#### Functions to complete:
1. `viterbi()`

#### Hint:
In order to reconstruct your most-likely path after running Viterbi, you'll need to keep track of a back-pointer at each state, which directs you to that state's most-likely predecessor.

You are asked to use the provided function `gaussian_prob` to compute  emission probabilities. Although in real work, you have to convert the probability to log-base in order to prevent digit underflow, in this assignment, we will only test your function against a rather short sequence of observations, so **DO NOT** convert the probability to logarithmic probability, otherwise you will fail the tests on Gradescope.

In the autograder, we will also test your code against other `evidence_vectors`.

In [ ]:
#export
def gaussian_prob(x, para_tuple):
    """Compute the probability of a given x value

    Args:
        x (float): observation value
        para_tuple (tuple): contains two elements, (mean, standard deviation)

    Return:
        Probability of seeing a value "x" in a Gaussian distribution.

    Note:
        We simplify the problem so you don't have to take care of integrals.
        Theoretically speaking, the returned value is not a probability of x,
        since the probability of any single value x from a continuous 
        distribution should be zero, instead of the number outputed here.
        By definition, the Gaussian percentile of a given value "x"
        is computed based on the "area" under the curve, from left-most to x. 
        The proability of getting value "x" is zero bcause a single value "x"
        has zero width, however, the probability of a range of value can be
        computed, for say, from "x - 0.1" to "x + 0.1".

    """
    if list(para_tuple) == [None, None]:
        return 0.0

    mean, std = para_tuple
    gaussian_percentile = (2 * np.pi * std**2)**-0.5 * \
                          np.exp(-(x - mean)**2 / (2 * std**2))
    return gaussian_percentile


In [ ]:
#export
def viterbi(evidence_vector, states, prior_probs,
            transition_probs, emission_paras):
    """Viterbi Algorithm to calculate the most likely states give the evidence.
    Args:
        evidence_vector (list): List of right hand Y-axis positions (interger).
        states (list): List of all states in a word. No transition between words.
                       example: ['B1', 'B2', 'B3', 'Bend', 'H1', 'H2', 'H3', 'Hend']
        prior_probs (dict): prior distribution for each state.
                            example: {'X1': 0.25,
                                      'X2': 0.25,
                                      'X3': 0.25,
                                      'Xend': 0.25}
        transition_probs (dict): dictionary representing transitions from each
                                 state to every other valid state such as for the above 
                                 states, there won't be a transition from 'B1' to 'H1'
        emission_paras (dict): parameters of Gaussian distribution 
                                from each state.
    Return:
        tuple of
        ( A list of states the most likely explains the evidence,
          probability this state sequence fits the evidence as a float )
    Note:
        You are required to use the function gaussian_prob to compute the
        emission probabilities.
    """
    
    # TODO: complete this function.
    raise NotImplementedError()

    sequence = []
    probability = 0.0

    return sequence, probability

########## DON'T WRITE ANY CODE OUTSIDE THE FUNCTION! ################
##### CODE BELOW IS USED FOR RUNNING LOCAL TEST DON'T MODIFY IT ######
tests.TestPart1b().test_viterbi_case1(part_1_a, viterbi)
tests.TestPart1b().test_viterbi_case2(part_1_a, viterbi)
tests.TestPart1b().test_viterbi_case3(part_1_a, viterbi)
tests.TestPart1b().test_viterbi_realsample1(part_1_a, viterbi)
tests.TestPart1b().test_viterbi_realsample2(part_1_a, viterbi)
tests.TestPart1b().test_viterbi_realsample3(part_1_a, viterbi)
################ END OF LOCAL TEST CODE SECTION ######################


### Part2a: Multidimensional Output Probabilities
_[6 Points]_

In Part 1a, we use only right-hand Y-axis coordinates as our solely feature, now we are going to use both hands. Since sign language is two handed, using features both to the right and left hands can increase the accuracy of our model when dealing with more complex sentences.

Here you are given with the transition probabilities, and the means & standard deviations for emission probabilties of left-hand Y-axis locations, following the same procedure conducted in Part 1a.

One thing to notice is, in Part 1, the `viterbi` function is tested against single words. In other words, the input evidence vector will not transit between different words. However, for Part 2, the input evidence vector can be either a single word, or a verb phrase such as "BUY CAR" and "BUY HOUSE". Adjust the probabilities in the image below to adapt to this fact. Note that consecutive words should be different.

<img src="part_2_a_probs.png" alt="2a_probs">

BUY | State 1 | State 2 | State 3
--- | --- | --- | --- 
Mean | 108.200 | 78.670 | 64.182
Std | 17.314 | 1.886 | 5.573

CAR | State 1 | State 2 | State 3
--- | --- | --- | --- 
Mean | 56.300 | 37.110 | 50.000
Std | 10.659 | 4.306 | 7.826

HOUSE | State 1 | State 2 | State 3
--- | --- | --- | --- 
Mean | 53.600 | 37.168 | 74.176
Std | 7.392 | 8.875 | 8.347

#### Functions to complete:
1. `part_2_a()`


In [ ]:
#export
def part_2_a():
    """Provide probabilities for the word HMMs outlined below.
    Now, at each time frame you are given with 2 observations (right hand Y
    position & left hand Y position). Use the result you derived in
    part_1_a, accompany with the provided probability for left hand, create
    a tuple of (right-y, left-y) to represent high-dimension transition & 
    emission probabilities.
    """

     # TODO: complete this function.
    raise NotImplementedError()

    """Word BUY"""
    b_prior_probs = {
        'B1': 0.,
        'B2': 0.,
        'B3': 0.,
        'Bend': 0.,
    }
    # example: {'B1': {'B1' : (right-hand Y, left-hand Y), ... }
    b_transition_probs = {
        'B1': {'B1': (0., 0.), 'B2': (0., 0.), 'B3': (0., 0.), 'Bend': (0., 0.)},
        'B2': {'B1': (0., 0.), 'B2': (0., 0.), 'B3': (0., 0.), 'Bend': (0., 0.)},
        'B3': {'B1': (0., 0.), 'B2': (0., 0.), 'B3': (0., 0.), 'Bend': (0., 0.)},
        'Bend': {'B1': (0., 0.), 'B2': (0., 0.), 'B3': (0., 0.), 'Bend': (0., 0.)},
    }
    # example: {'B1': [(right-mean, right-std), (left-mean, left-std)] ...}
    b_emission_paras = {
        'B1': [(None, None), (None, None)],
        'B2': [(None, None), (None, None)],
        'B3': [(None, None), (None, None)],
        'Bend': [(None, None), (None, None)]
    }

    """Word Car"""
    c_prior_probs = {
        'C1': 0.,
        'C2': 0.,
        'C3': 0.,
        'Cend': 0.,
    }
    c_transition_probs = {
        'C1': {'C1': (0., 0.), 'C2': (0., 0.), 'C3': (0., 0.), 'Cend': (0., 0.)},
        'C2': {'C1': (0., 0.), 'C2': (0., 0.), 'C3': (0., 0.), 'Cend': (0., 0.)},
        'C3': {'C1': (0., 0.), 'C2': (0., 0.), 'C3': (0., 0.), 'Cend': (0., 0.)},
        'Cend': {'C1': (0., 0.), 'C2': (0., 0.), 'C3': (0., 0.), 'Cend': (0., 0.)},
    }
    c_emission_paras = {
        'C1': [(None, None), (None, None)],
        'C2': [(None, None), (None, None)],
        'C3': [(None, None), (None, None)],
        'Cend': [(None, None), (None, None)]
    }

    """Word HOUSE"""
    h_prior_probs = {
        'H1': 0.,
        'H2': 0.,
        'H3': 0.,
        'Hend': 0.,
    }
    h_transition_probs = {
        'H1': {'H1': (0., 0.), 'H2': (0., 0.), 'H3': (0., 0.), 'Hend': (0., 0.)},
        'H2': {'H1': (0., 0.), 'H2': (0., 0.), 'H3': (0., 0.), 'Hend': (0., 0.)},
        'H3': {'H1': (0., 0.), 'H2': (0., 0.), 'H3': (0., 0.), 'Hend': (0., 0.)},
        'Hend': {'H1': (0., 0.), 'H2': (0., 0.), 'H3': (0., 0.), 'Hend': (0., 0.)},
    }
    h_emission_paras = {
        'H1': [(None, None), (None, None)],
        'H2': [(None, None), (None, None)],
        'H3': [(None, None), (None, None)],
        'Hend': [(None, None), (None, None)]
    }

    return (b_prior_probs, b_transition_probs, b_emission_paras,
            c_prior_probs, c_transition_probs, c_emission_paras,
            h_prior_probs, h_transition_probs, h_emission_paras,)

########## DON'T WRITE ANY CODE OUTSIDE THE FUNCTION! ################
##### CODE BELOW IS USED FOR RUNNING LOCAL TEST DON'T MODIFY IT ######
tests.TestPart2a().test_prior(part_2_a)
tests.TestPart2a().test_b_emission(part_2_a)
tests.TestPart2a().test_c_emission(part_2_a)
tests.TestPart2a().test_h_emission(part_2_a)
tests.TestPart2a().test_b_transition(part_2_a)
tests.TestPart2a().test_c_transition(part_2_a)
tests.TestPart2a().test_h_transition(part_2_a)
################ END OF LOCAL TEST CODE SECTION ######################

### Part 2b: Improving the Viterbi Trellis
_[39 Points]_

Modify the Viterbi Trellis function to allow multiple observed values (Y location of right and left hands) for a state. The return format should be identical to Part 1b.


#### Functions to complete:
1. `multidimensional_viterbi()`


In [ ]:
#export
def multidimensional_viterbi(evidence_vector, states, prior_probs,
                             transition_probs, emission_paras):
    """Decode the most likely word phrases generated by the evidence vector.
    States, prior_probs, transition_probs, and emission_probs will now contain
    all the words from part_2_a.
    Evidence vector is a list of tuples where the first element of each tuple is the right 
    hand coordinate and the second element is the left hand coordinate.
    """
    # TODO: complete this function.
    raise NotImplementedError()

    sequence = []
    probability = 0.0

    return sequence, probability

########## DON'T WRITE ANY CODE OUTSIDE THE FUNCTION! ################
##### CODE BELOW IS USED FOR RUNNING LOCAL TEST DON'T MODIFY IT ######
tests.TestPart2b().test_viterbi_case1(part_2_a, multidimensional_viterbi)
tests.TestPart2b().test_viterbi_case2(part_2_a, multidimensional_viterbi)
tests.TestPart2b().test_viterbi_case3(part_2_a, multidimensional_viterbi)
tests.TestPart2b().test_viterbi_realsample1(part_2_a, multidimensional_viterbi)
tests.TestPart2b().test_viterbi_realsample2(part_2_a, multidimensional_viterbi)
tests.TestPart2b().test_viterbi_realsample3(part_2_a, multidimensional_viterbi)
tests.TestPart2b().test_viterbi_phrase1(part_2_a, multidimensional_viterbi)
tests.TestPart2b().test_viterbi_phrase2(part_2_a, multidimensional_viterbi)
################ END OF LOCAL TEST CODE SECTION ######################

In [ ]:
#export
def return_your_name():
    """Return your name
    """
    # TODO: finish this
    raise NotImplementedError()
    
    return ""

**CONGRATULATIONS!**  You have just completed your final assignment for CS6601 Artificial Intelligence.


**CONGRATULATIONS!**  You have just completed your final assignment for CS6601 Artificial Intelligence.


### Bonus
_[5 Points]_

Note: All graphics are from Dr. Ploetz's 2005 Disseration ["Advanced Stochastic Protein Sequence Analysis"](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.667.139&rep=rep1&type=pdf)

#### Description

You might have noticed that the HMMs that you've constructed in this assignment are created and trained using data from only one user in a static environment. While these can be useful in practice, it is often the case that we want our HMMS to adapt to many different users and many different environments. For example, if we're reading hand positions from a video frame, what if the capture device is tilted? Our left and right hand position data will be affected, and our HMM could output a low probability for what in reality could be a perfect sign. How do we approach this problem?

One solution is to collect lots of data from whatever scenario we find ourselves in and train an entirely new HMM. This is infeasible in many real-world applications, and is extremely data inefficient as we discard all data collected beforehand. A more practical solution is to collect a small amount of data for our new scenario, and use this data to adapt our previously trained HMM for use in this unfamiliar environment. One prevalent technique to apply this idea is called MLLR, or Maximum Likelihood Linear Regression, and involves transforming Gaussian parameters in an HMM based on new adaptation data. It is feasible in real applications as it only requires a small amount of adaptation data, and is data efficient in that we keep the information gained from our original training data. In this bonus, you'll adapt your constructed models to a new user.

In the context of sign language and many others, when we adapt our model we don't want to change the transition probabilities, only the gaussian emission parameters of each state. Think of it as the signs themselves are not changing; simply the context in which we observe our data is changing. If we plotted all of our emission gaussians for all states, we might see something like this:

<img src="bonus_graphics/sign_mixture.PNG" alt="gaussian_plot">

In the scenario above with the tilted capture device, we would expect the general form of our model to stay the same with a slight transformation to fit the misaligned angle. Given a small amount of data from this tilted perspective, we would want to construct a transformation such that we can produce a HMM for this specific scenario. This is depicted in the graphic below:

<img src="bonus_graphics/adapted_signs.PNG" alt="gaussian_plot">

##### In this bonus we'll focus on creating a transformation matrix to change the means of each gaussian, not the variance.


#### Procedure

Below are three new data sequences for a new user in an unknown environment. Note that the data points in each state are provided, and there is no need for moving data points between states.

Word | Frames | Observed sequence (R, L) | State1 | State2 | State3
--- | --- | --- | --- | --- | --- 
BUY | 11 | (61,123), (61, 116), (59, 121), (65, 99), (73, 97), (75, 98), (79, 74), (79, 84), (79, 84), (74, 89), (68, 81) | (61,123), (61, 116), (59, 121) | (65, 99), (73, 97), (75, 98) | (79, 74), (79, 84), (79, 84), (74, 89), (68, 81) 
CAR | 8 | (44, 73), (53, 70), (62, 78), (64, 62), (66, 58), (59, 51), (61, 76), (58, 90)| (44, 73), (53, 70), (62, 78)| (64, 62), (66, 58), (59, 51)| (61, 76), (58, 90)
HOUSE | 16 | (59, 65), (59, 68), (60, 69), (57, 70), (56, 64), (49, 59), (51, 57), (51, 51), (53, 51), (59, 59), (72, 79), (81, 82), (82, 89), (84, 90), (86, 90), (90, 93)| (59, 65), (59, 68), (60, 69), (57, 70), (56, 64)| (49, 59), (51, 57), (51, 51), (53, 51), (59, 59)| (72, 79), (81, 82), (82, 89), (84, 90), (86, 90), (90, 93)


The original proposal of this technique is described in [C.J. Leggetter's 1995 paper](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.89.2050&rep=rep1&type=pdf), and we'll outline the steps here. Our objective is to construct a transformation matrix $W$ that we can apply to our existing means to produce the new parameters for our transformed HMM:

<img src="bonus_graphics/adapted_means.PNG" alt="gaussian_plot">

We can find this matrix W by the following equations:

<img src="bonus_graphics/equations.PNG" alt="gaussian_plot">

#### Variable Explanation:

$R$: There are many derivations and considerations that we're simplifying in this section, and one of them is Regression Classes, denoted by $R$ in the above equations. A Regression Class is used if you want your adaptation data to only apply to a subset of your HMM, or only a certain collection of states. For this bonus, we consider all our states to be in the same Regression Class, and thus $\delta$ will always be equal to one.

$\vec{x}_t$ : $\vec{x}$ is the adaptation data vector, and $\vec{x}_t$ represents the data collected in time-step $t$.

$s_t$: $s_t$ represents the state that the sequence is currently in at time-step $t$.

$\mu_{ki}$: $\mu_k$ represents the means of a particular state $k$, and $i$ refers to each dimension in the mean.  

#### Functions to complete:

1. `MLLR_results()`

In [ ]:
#export
def MLLR_results():
    
    """Complete the MLLR adaptation process with the new adaptation data and 
     return the new emission parameters for each state.
    """
    # TODO: complete this function.


    b_emission_paras = {
        'B1': [(None, None), (None, None)],
        'B2': [(None, None), (None, None)],
        'B3': [(None, None), (None, None)],
        'Bend': [(None, None), (None, None)]
    }

    c_emission_paras = {
        'C1': [(None, None), (None, None)],
        'C2': [(None, None), (None, None)],
        'C3': [(None, None), (None, None)],
        'Cend': [(None, None), (None, None)]
    }

    h_emission_paras = {
        'H1': [(None, None), (None, None)],
        'H2': [(None, None), (None, None)],
        'H3': [(None, None), (None, None)],
        'Hend': [(None, None), (None, None)]
    }

    return (b_emission_paras, 
            c_emission_paras, 
            h_emission_paras)